💻 **준비 코드**

In [1]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('gender_submission.csv')

<br>

# 3. 첫 번째 생존자 예측

앞서 우리는 수치형 변수들과 생존율의 관계를 분석했습니다. 1등석 승객의 생존율이 63%로 가장 높았고, 비싼 요금을 낸 승객일수록 생존 확률이 높았습니다. 이제 이러한 통찰을 바탕으로 첫 번째 예측 모델을 만들어보겠습니다.

## 학습 과정 개요

머신러닝 모델을 만드는 과정은 다음과 같은 단계로 이루어집니다:

1. 학습에 사용할 변수 선택
   - 앞서 분석한 수치형 변수들 중에서 선택
   - 문자형 변수들은 다음 장에서 다룰 예정
2. 데이터 분할 (8:2 비율)
   - 학습용 데이터(80%)로 패턴을 학습
   - 검증용 데이터(20%)로 학습이 잘 되었는지 확인
3. 모델 학습
   - Random Forest 알고리즘 사용
   - 여러 전문가의 의견을 종합하는 것과 같은 방식
4. 생존 여부 예측
   - 학습된 모델로 test.csv의 승객 생존 여부 예측
   - 예측 결과를 캐글에 제출

:::{note}

**이 섹션이 필요한 이유** 

이번 섹션에서는 다소 복잡한 코드들이 등장합니다. 하지만 걱정하지 마세요! 지금 이 코드들을 완벽히 이해할 필요는 없습니다.

자동차를 처음 운전하는 사람이 엔진의 작동 원리를 완벽히 이해하지 않아도 운전할 수 있는 것처럼, 여러분도 이 코드들의 세부적인 내용을 모두 이해하지 않아도 예측 모델을 만들 수 있습니다.

지금은 "이런 과정을 통해 인공지능이 학습을 하는구나" 정도로 이해하시면 충분합니다. 각 단계가 어떤 의미인지는 이후 챕터들에서 차근차근 설명드리겠습니다.

:::



## 학습에 사용할 변수 선택

먼저 모델 학습에 사용할 변수들을 선택하겠습니다. 1.2 섹션에서 분석한 수치형 변수들 중에서 다음 다섯 가지를 사용하겠습니다:

- Pclass: 좌석 등급 (1등석, 2등석, 3등석)
- Age: 승객의 나이
- SibSp: 함께 탑승한 형제자매, 배우자의 수
- Parch: 함께 탑승한 부모, 자녀의 수
- Fare: 티켓 요금



📝 **프롬프트**
```
'Pclass', 'Age', 'SibSp', 'Parch', 'Fare' 변수들의 이름을 inc_fts 변수에 저장해줘
```

💻 **코드 & 실행결과**

In [2]:
inc_fts = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

이 다섯 가지 변수는 모두 생존과 어떤 형태로든 관련이 있을 것으로 예상됩니다. 예를 들어:
- 좌석 등급은 구명보트 탑승 우선순위와 관련이 있었을 것
- 나이는 비상 상황에서의 대처 능력과 연관
- 가족 관계는 도움을 받을 수 있는 가능성과 관련
- 요금은 객실의 위치나 서비스 수준을 반영

이제 이 변수들을 사용해 모델이 학습할 수 있는 형태로 데이터를 준비해 보겠습니다.



## 데이터 분할

이제 데이터를 컴퓨터가 학습하기 좋은 형태로 나누어보겠습니다. 마치 학생이 문제집으로 공부하고 모의고사로 실력을 확인하듯이, 데이터도 학습용과 검증용으로 나눕니다.

📝 **프롬프트**
```
1. 선택한 특성들은 X로, 생존여부는 y로 저장
2. test 데이터도 같은 특성들만 선택해서 X_test로 저장
3. 데이터 크기 확인
```

💻 **코드 & 실행결과**

In [3]:
X = train[inc_fts]     # 선택한 특성들
y = train['Survived']  # 생존 여부
X_test = test[inc_fts] # 예측할 데이터

print(X.shape, y.shape, X_test.shape)

(891, 5) (891,) (418, 5)


이렇게 분리된 데이터는 각각 다음과 같은 역할을 합니다:
- X: 선택한 5개 특성의 값들 (891명 × 5개 특성)
- y: 실제 생존 여부 (891명의 생존/사망 정보)
- X_test: 예측해야 할 418명의 승객 정보 (418명 × 5개 특성)

<br>

이제 X와 y 데이터를 다시 학습용과 검증용으로 나누어보겠습니다. 전체 데이터의 80%는 학습에 사용하고, 20%는 모델의 성능을 검증하는데 사용하겠습니다.

📝 **프롬프트**
```
X와 y를 8:2 비율로 나누어서 X_train, X_valid, y_train, y_valid로 저장해줘
```

💻 **코드 & 실행결과**

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

`train_test_split` 함수는 데이터를 무작위로 분할해주는 도구입니다. `random_state=42`는 매번 같은 방식으로 분할되도록 하는 설정으로, 실험의 재현성을 위해 추가했습니다.



## 모델 학습

이제 Random Forest 알고리즘을 사용해 모델을 학습시키겠습니다. Random Forest는 마치 100명의 전문가에게 의견을 물어보고, 다수결로 최종 결정을 내리는 것과 같습니다.

예를 들어, 한 승객의 생존 여부를 예측할 때:

- 전문가 A: "1등석이니까 생존했을 것" (나이는 고려하지 않음)
- 전문가 B: "나이가 어려서 생존했을 것" (좌석 등급은 고려하지 않음)
- 전문가 C: "요금이 비싸고 가족도 있어서 생존했을 것"

이런 식으로 각 전문가가 서로 다른 특성들에 주목하여 판단을 내리고, 최종적으로 가장 많은 전문가가 선택한 결과를 채택합니다.

📝 **프롬프트**
```
Random Forest 모델로 X_train과 y_train을 학습시켜줘
```

💻 **코드 & 실행결과**

In [5]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

<br>

이렇게 학습된 모델이 얼마나 잘 예측하는지 확인해볼 차례입니다. 앞서 우리는 전체 데이터의 20%를 검증용으로 따로 보관해두었죠. 이 데이터들의 생존 여부를 모델이 예측하도록 하고, 실제 값과 비교해보겠습니다.

📝 **프롬프트**
```
X_valid를 예측해서 y_pred에 저장하고, 실제 값(y_valid)과 비교해서 정확도를 계산해줘
```

💻 **코드 & 실행결과**

In [6]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print(f"Accuracy: {accuracy:.5f}")

Accuracy: 0.73184


우리 모델의 정확도는 73.184%로 나왔습니다. 이는 Validation Score로, 학습에 사용하지 않은 검증용 데이터에 대한 예측 정확도를 의미합니다. 구체적으로는:

1. 전체 train 데이터(891명) 중 20%인 약 178명을 검증용으로 따로 보관했었죠
2. 이 178명의 생존 여부를 모델이 예측해보았고
3. 그 중 73.184%인 약 130명의 생존 여부를 올바르게 맞혔다는 뜻입니다

Validation Score는 모델의 실제 성능을 가늠하는 중요한 지표입니다. 만약 학습에 사용한 데이터로 성능을 측정했다면, 마치 시험 문제를 외워서 푸는 것과 같아서 실제 실력을 평가하기 어렵기 때문입니다.

이 73.184%라는 점수는 나쁘지 않은 첫 시도라고 할 수 있습니다. 다음 단계에서는 이 모델을 실제 test 데이터에 적용하여 캐글에 제출해보도록 하겠습니다.



## 최종 예측 및 제출

이제 진짜 test 데이터에 대한 예측을 해보겠습니다. 이전의 검증용 데이터와는 달리, test 데이터는 실제 생존 여부를 모르는 상태에서 예측을 하는 것입니다.

📝 **프롬프트**
```
1. X_test에 대해 예측해서 결과를 submission 파일에 저장하고
2. 캐글 제출용 파일(titanic_pred.csv)로 내보내줘
```

💻 **코드 & 실행결과**

In [7]:
predictions = model.predict(X_test)
submission['Survived'] = predictions
submission.to_csv('titanic_pred.csv', index=False)

submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,0


이렇게 생성된 'titanic_pred.csv' 파일을 캐글에 제출한 결과, 0.63397의 점수를 받았습니다. 이는 test 데이터 418명 중 약 265명(418 × 0.63397 ≈ 265)의 생존 여부를 정확하게 예측했다는 의미입니다.

```{figure} images/13-1.png
---
width: 600px
---
결과파일 제출
```



지금까지의 결과를 표로 정리하면 다음과 같습니다:

| 버전 | 피쳐 개수 | Val. Score | Public Score | 맞은 사람 수 |          설명           |
| :--: | :-------: | :--------: | :----------: | :----------: | :---------------------: |
| 1.3  |     5     |  0.73184   |   0.63397    |    265명     | 5개의 numeric 피쳐 사용 |

이 점수는 이전의 성별 기반 예측보다 낮습니다. 그 이유는 현재 모델이 숫자형(Numeric) 변수들만을 사용했기 때문입니다. 성별(Sex)과 같은 중요한 정보들이 문자형(Object) 변수라서 아직 활용하지 못했죠.

하지만 걱정하지 마세요! 이것은 우리의 첫 번째 시도일 뿐입니다. 앞으로 다음과 같은 방법들을 통해 점수를 높여갈 것입니다:

1. 문자형 변수들을 활용하는 방법 배우기
2. 결측치를 더 효과적으로 처리하기
3. 변수들간의 관계를 활용하기
4. 다양한 머신러닝 알고리즘 시도해보기

다음 챕터에서는 특히 성별(Sex)과 승선 항구(Embarked)라는 문자형 변수들을 어떻게 활용할 수 있는지 배워보도록 하겠습니다. 이를 통해 우리의 예측 모델이 얼마나 개선되는지 함께 확인해보시죠!